In [16]:
!pip install langchain==0.3.30
!pip install langchain==0.3.14
!pip install langchain-openai==0.3.0
!pip install langchain-community==0.3.14
!pip install langchain-huggingface==0.1.2

ERROR: Could not find a version that satisfies the requirement langchain==0.3.30 (from versions: 0.0.1, 0.0.2, 0.0.3, 0.0.4, 0.0.5, 0.0.6, 0.0.7, 0.0.8, 0.0.9, 0.0.10, 0.0.11, 0.0.12, 0.0.13, 0.0.14, 0.0.15, 0.0.16, 0.0.17, 0.0.18, 0.0.19, 0.0.20, 0.0.21, 0.0.22, 0.0.23, 0.0.24, 0.0.25, 0.0.26, 0.0.27, 0.0.28, 0.0.29, 0.0.30, 0.0.31, 0.0.32, 0.0.33, 0.0.34, 0.0.35, 0.0.36, 0.0.37, 0.0.38, 0.0.39, 0.0.40, 0.0.41, 0.0.42, 0.0.43, 0.0.44, 0.0.45, 0.0.46, 0.0.47, 0.0.48, 0.0.49, 0.0.50, 0.0.51, 0.0.52, 0.0.53, 0.0.54, 0.0.55, 0.0.56, 0.0.57, 0.0.58, 0.0.59, 0.0.60, 0.0.61, 0.0.63, 0.0.64, 0.0.65, 0.0.66, 0.0.67, 0.0.68, 0.0.69, 0.0.70, 0.0.71, 0.0.72, 0.0.73, 0.0.74, 0.0.75, 0.0.76, 0.0.77, 0.0.78, 0.0.79, 0.0.80, 0.0.81, 0.0.82, 0.0.83, 0.0.84, 0.0.85, 0.0.86, 0.0.87, 0.0.88, 0.0.89, 0.0.90, 0.0.91, 0.0.92, 0.0.93, 0.0.94, 0.0.95, 0.0.96, 0.0.97, 0.0.98, 0.0.99rc0, 0.0.99, 0.0.100, 0.0.101rc0, 0.0.101, 0.0.102rc0, 0.0.102, 0.0.103, 0.0.104, 0.0.105, 0.0.106, 0.0.107, 0.0.108, 0.0.109, 0.0

In [54]:
from langchain import HuggingFaceHub
from langchain_huggingface import ChatHuggingFace
from huggingface_hub import login
from getpass import getpass

In [55]:
hf_api = getpass("enter your api key")

enter your api key··········


In [56]:
login(hf_api)

In [57]:
llm = HuggingFaceHub(
    repo_id = "google/gemma-1.1-2b-it",
    model_kwargs = {"temparature": 0.5 , "repitation_penalty": 1.03, "max_new_tokens":200 , "max_length":180},
    huggingfacehub_api_token = hf_api)

In [58]:
model = ChatHuggingFace(llm = llm)

In [62]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

In [63]:
response_schemas = [
    ResponseSchema(name = "answer", description = "answer the user's questions"),
    ResponseSchema(name = "source", description = "provide the full `source` for answer to the users question which should be `website URL` ")
]

In [65]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [66]:
output_parser

StructuredOutputParser(response_schemas=[ResponseSchema(name='answer', description="answer the user's questions", type='string'), ResponseSchema(name='source', description='provide the full `source` for answer to the users question which should be `website URL` ', type='string')])

In [67]:
from langchain_core.prompts import PromptTemplate

In [69]:
format_instructions = output_parser.get_format_instructions()

In [70]:
prompt = PromptTemplate(
    template = "answer the user's question as best as possible \n{format_instructions} \n {question}",
    input_variables= ["question"],
    partial_variables= {"format_instructions": format_instructions}
)

In [71]:
prompt

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={'format_instructions': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"answer": string  // answer the user\'s questions\n\t"source": string  // provide the full `source` for answer to the users question which should be `website URL` \n}\n```'}, template="answer the user's question as best as possible \n{format_instructions} \n {question}")

In [86]:
chain = prompt | model

In [87]:
chain.invoke({"question":"what is the most popular food item in Hyderabad"}).content

'<bos><start_of_turn>user\nanswer the user\'s question as best as possible \nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"answer": string  // answer the user\'s questions\n\t"source": string  // provide the full `source` for answer to the users question which should be `website URL` \n}\n``` \n what is the most popular food item in Hyderabad<end_of_turn>\n<start_of_turn>model\n```json\n{\n\t"answer": "Hyderabadi Biryani",  // answer the user\'s questions\n\t"source": "https://www.tasteofhyderabad.com/most-popular-hyderabad-food/"  // provide the full source for answer to the users question which should be website URL\n}\n```'

In [88]:
for i in chain.stream({"question": "what is the best food item in vizag"}):
  print(i.content)

<bos><start_of_turn>user
answer the user's question as best as possible 
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": string  // answer the user's questions
	"source": string  // provide the full `source` for answer to the users question which should be `website URL` 
}
``` 
 what is the best food item in vizag<end_of_turn>
<start_of_turn>model
```json
{
	"answer": "The best food item in Vizag is undoubtedly **seafood**. The region boasts a rich coastline with diverse marine life, making it a paradise for seafood lovers. From fresh catches like crab, prawns, and lobsters to succulent fish dishes and seafood platters, Vizag offers an endless array of culinary delights.",
	"source": "https://yourwebsite.com/food-in-vizag"  // Replace with the actual website URL
}
```


In [89]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

# Response to the user's question
response_schemas = [
    ResponseSchema(name="answer", description="Answer to the user's question"),
    ResponseSchema(
        name="source",
        description="The `source` used to answer the user's question, which should be a `website URL`.",
    ),
]
# Initialize the structured output parser based on the response schemas
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [90]:
from langchain_core.prompts import PromptTemplate
# Parse the format instructions.
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    # Set up the template to answer the user's question as best as possible.
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    # Use 'question' as the input variable.
    input_variables=["question"],
    # Use 'format_instructions' as a partial variable.
    partial_variables={"format_instructions": format_instructions},
)

In [92]:
chain = prompt | model   # Connect the prompt, model, and output parser

# Ask the question, "What is the largest desert in the world?"
chain.invoke({"question": "What is the largest desert in the world?"})

AIMessage(content='<bos><start_of_turn>user\nanswer the users question as best as possible.\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"answer": string  // Answer to the user\'s question\n\t"source": string  // The `source` used to answer the user\'s question, which should be a `website URL`.\n}\n```\nWhat is the largest desert in the world?<end_of_turn>\n<start_of_turn>model\n```json\n{\n\t"answer": "Sahara Desert",  // Answer to the user\'s question\n\t"source": "https://en.wikipedia.org/wiki/Sahara_Desert"  // The source used to answer the user\'s question, which is Wikipedia.\n}\n```', additional_kwargs={}, response_metadata={}, id='run-62a3e051-196e-49e0-bafd-8ba90c329151-0')